In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

2022-06-14 15:49:01.326867: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-06-14 15:49:01.326911: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['text']:
        training_sentences.append(pattern)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])
    
    if intent['intent'] not in labels:
        labels.append(intent['intent'])
        
num_classes = len(labels)

In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
Total params: 16,952
Trainable params: 16,952
Non-trainable params: 0
_________________________________________________________________


2022-06-14 15:49:02.396626: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-14 15:49:02.396641: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-14 15:49:02.396653: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (luisfgbs-HP-ENVY-x360-Convertible-15-es0xxx): /proc/driver/nvidia/version does not exist
2022-06-14 15:49:02.396798: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-14 15:49:02.424927: I tensorflow/core/platform/profile_utils/cpu

In [6]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
6/6 [==============================] - 0s 705us/step - loss: 3.1785 - accuracy: 0.0349
Epoch 2/500
6/6 [==============================] - 0s 562us/step - loss: 3.1753 - accuracy: 0.0640
Epoch 3/500
6/6 [==============================] - 0s 793us/step - loss: 3.1720 - accuracy: 0.2151
Epoch 4/500
6/6 [==============================] - 0s 677us/step - loss: 3.1694 - accuracy: 0.1570
Epoch 5/500
6/6 [==============================] - 0s 572us/step - loss: 3.1662 - accuracy: 0.1163
Epoch 6/500
6/6 [==============================] - 0s 553us/step - loss: 3.1630 - accuracy: 0.1163
Epoch 7/500
6/6 [==============================] - 0s 535us/step - loss: 3.1590 - accuracy: 0.1163
Epoch 8/500
6/6 [==============================] - 0s 722us/step - loss: 3.1551 - accuracy: 0.1163
Epoch 9/500
6/6 [==============================] - 0s 721us/step - loss: 3.1501 - accuracy: 0.1163
Epoch 10/500
6/6 [==============================] - 0s 605us/step - loss: 3.1447 - accuracy: 0.1163
Epoch 11/

6/6 [==============================] - 0s 600us/step - loss: 2.0232 - accuracy: 0.4884
Epoch 84/500
6/6 [==============================] - 0s 604us/step - loss: 1.9992 - accuracy: 0.4884
Epoch 85/500
6/6 [==============================] - 0s 787us/step - loss: 1.9779 - accuracy: 0.4884
Epoch 86/500
6/6 [==============================] - 0s 895us/step - loss: 1.9545 - accuracy: 0.4884
Epoch 87/500
6/6 [==============================] - 0s 809us/step - loss: 1.9321 - accuracy: 0.5000
Epoch 88/500
6/6 [==============================] - 0s 590us/step - loss: 1.9099 - accuracy: 0.5058
Epoch 89/500
6/6 [==============================] - 0s 503us/step - loss: 1.8845 - accuracy: 0.5174
Epoch 90/500
6/6 [==============================] - 0s 662us/step - loss: 1.8621 - accuracy: 0.5291
Epoch 91/500
6/6 [==============================] - 0s 601us/step - loss: 1.8355 - accuracy: 0.5291
Epoch 92/500
6/6 [==============================] - 0s 716us/step - loss: 1.8096 - accuracy: 0.5349
Epoch 93/500


6/6 [==============================] - 0s 800us/step - loss: 0.6465 - accuracy: 0.8547
Epoch 165/500
6/6 [==============================] - 0s 700us/step - loss: 0.6390 - accuracy: 0.8547
Epoch 166/500
6/6 [==============================] - 0s 780us/step - loss: 0.6327 - accuracy: 0.8547
Epoch 167/500
6/6 [==============================] - 0s 755us/step - loss: 0.6214 - accuracy: 0.8605
Epoch 168/500
6/6 [==============================] - 0s 840us/step - loss: 0.6126 - accuracy: 0.8605
Epoch 169/500
6/6 [==============================] - 0s 598us/step - loss: 0.6041 - accuracy: 0.8605
Epoch 170/500
6/6 [==============================] - 0s 624us/step - loss: 0.5959 - accuracy: 0.8547
Epoch 171/500
6/6 [==============================] - 0s 885us/step - loss: 0.5898 - accuracy: 0.8547
Epoch 172/500
6/6 [==============================] - 0s 686us/step - loss: 0.5826 - accuracy: 0.8430
Epoch 173/500
6/6 [==============================] - 0s 863us/step - loss: 0.5735 - accuracy: 0.8430
Epoc

6/6 [==============================] - 0s 499us/step - loss: 0.2352 - accuracy: 0.9709
Epoch 246/500
6/6 [==============================] - 0s 763us/step - loss: 0.2324 - accuracy: 0.9767
Epoch 247/500
6/6 [==============================] - 0s 560us/step - loss: 0.2302 - accuracy: 0.9767
Epoch 248/500
6/6 [==============================] - 0s 665us/step - loss: 0.2267 - accuracy: 0.9767
Epoch 249/500
6/6 [==============================] - 0s 531us/step - loss: 0.2229 - accuracy: 0.9826
Epoch 250/500
6/6 [==============================] - 0s 516us/step - loss: 0.2207 - accuracy: 0.9767
Epoch 251/500
6/6 [==============================] - 0s 539us/step - loss: 0.2184 - accuracy: 0.9651
Epoch 252/500
6/6 [==============================] - 0s 571us/step - loss: 0.2153 - accuracy: 0.9767
Epoch 253/500
6/6 [==============================] - 0s 745us/step - loss: 0.2133 - accuracy: 0.9826
Epoch 254/500
6/6 [==============================] - 0s 540us/step - loss: 0.2106 - accuracy: 0.9767
Epoc

6/6 [==============================] - 0s 624us/step - loss: 0.0893 - accuracy: 1.0000
Epoch 327/500
6/6 [==============================] - 0s 1ms/step - loss: 0.0882 - accuracy: 1.0000
Epoch 328/500
6/6 [==============================] - 0s 516us/step - loss: 0.0872 - accuracy: 1.0000
Epoch 329/500
6/6 [==============================] - 0s 855us/step - loss: 0.0864 - accuracy: 1.0000
Epoch 330/500
6/6 [==============================] - 0s 933us/step - loss: 0.0855 - accuracy: 1.0000
Epoch 331/500
6/6 [==============================] - 0s 833us/step - loss: 0.0841 - accuracy: 1.0000
Epoch 332/500
6/6 [==============================] - 0s 1ms/step - loss: 0.0835 - accuracy: 1.0000
Epoch 333/500
6/6 [==============================] - 0s 783us/step - loss: 0.0823 - accuracy: 1.0000
Epoch 334/500
6/6 [==============================] - 0s 1ms/step - loss: 0.0820 - accuracy: 1.0000
Epoch 335/500
6/6 [==============================] - 0s 728us/step - loss: 0.0811 - accuracy: 1.0000
Epoch 336/

6/6 [==============================] - 0s 695us/step - loss: 0.0368 - accuracy: 1.0000
Epoch 408/500
6/6 [==============================] - 0s 890us/step - loss: 0.0364 - accuracy: 1.0000
Epoch 409/500
6/6 [==============================] - 0s 573us/step - loss: 0.0361 - accuracy: 1.0000
Epoch 410/500
6/6 [==============================] - 0s 667us/step - loss: 0.0359 - accuracy: 1.0000
Epoch 411/500
6/6 [==============================] - 0s 618us/step - loss: 0.0354 - accuracy: 1.0000
Epoch 412/500
6/6 [==============================] - 0s 483us/step - loss: 0.0350 - accuracy: 1.0000
Epoch 413/500
6/6 [==============================] - 0s 1ms/step - loss: 0.0347 - accuracy: 1.0000
Epoch 414/500
6/6 [==============================] - 0s 543us/step - loss: 0.0343 - accuracy: 1.0000
Epoch 415/500
6/6 [==============================] - 0s 571us/step - loss: 0.0343 - accuracy: 1.0000
Epoch 416/500
6/6 [==============================] - 0s 774us/step - loss: 0.0336 - accuracy: 1.0000
Epoch 

6/6 [==============================] - 0s 719us/step - loss: 0.0173 - accuracy: 1.0000
Epoch 489/500
6/6 [==============================] - 0s 829us/step - loss: 0.0171 - accuracy: 1.0000
Epoch 490/500
6/6 [==============================] - 0s 573us/step - loss: 0.0170 - accuracy: 1.0000
Epoch 491/500
6/6 [==============================] - 0s 506us/step - loss: 0.0169 - accuracy: 1.0000
Epoch 492/500
6/6 [==============================] - 0s 719us/step - loss: 0.0167 - accuracy: 1.0000
Epoch 493/500
6/6 [==============================] - 0s 928us/step - loss: 0.0167 - accuracy: 1.0000
Epoch 494/500
6/6 [==============================] - 0s 833us/step - loss: 0.0164 - accuracy: 1.0000
Epoch 495/500
6/6 [==============================] - 0s 702us/step - loss: 0.0164 - accuracy: 1.0000
Epoch 496/500
6/6 [==============================] - 0s 503us/step - loss: 0.0162 - accuracy: 1.0000
Epoch 497/500
6/6 [==============================] - 0s 861us/step - loss: 0.0161 - accuracy: 1.0000
Epoc

In [7]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2022-06-14 15:49:06.240562: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets
